# 빅프9조

 필수 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


상수 선언

In [2]:
NS_DIFF_1M = 0.000007
EW_DIFF_1M = 0.000009
PATROL_HR = 900
GRID_DIFF = 90
data_path = "data/"
encoding = "utf-8-sig"상수 선언
#상계1
police = np.array([37.679699, 127.055066], dtype=np.float64)

원형으로 필터링

In [3]:
def filtering(police, grid):
    ft = (grid - police).copy()
    ft[:,0] /= NS_DIFF_1M
    ft[:,1] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    grid = grid[ft <= PATROL_HR + 10, :]
    return grid

격자점 반환

In [4]:
def grids(police):
    hcount = PATROL_HR//GRID_DIFF
    ew = np.arange(-hcount, hcount + 1, dtype=np.float64)
    ns = ew.reshape(-1,1).copy()
    ew *= EW_DIFF_1M * GRID_DIFF
    ns *= NS_DIFF_1M * GRID_DIFF
    
    result = np.full((2*hcount+1, 2*hcount+1, 2), police, dtype=np.float64)
    result[:,:,0] += ns
    result[:,:,1] += ew
    
    return filtering(police, result.reshape(-1,2))

In [5]:
def draw_marker(m, grid):
    for i in grid:
        folium.CircleMarker(location=i, radius=10, color='#000000').add_to(m)

In [6]:
m = folium.Map(location = police, zoom_start=60)
folium.Marker(location=police, 
              popup='지구대',
              icon=folium.Icon(color='lightblue', icon='star')
).add_to(m)

In [7]:
police_grid = grids(police)
draw_marker(m, police_grid)

In [8]:
m

In [9]:
SAMPLING = 5

In [10]:
def sampled_grids(grid):
    interval_size = SAMPLING+1
    result = np.zeros((grid.shape[0]*interval_size, 2), dtype=np.float64)
    for i in range(0, result.shape[0], interval_size):
        ran = np.zeros((interval_size, 2), dtype=np.float64)
        ran[1:] = np.random.uniform(-1, 1, (interval_size-1, 2))
        ran[:, 0] *= (GRID_DIFF/2)*NS_DIFF_1M
        ran[:, 1] *= (GRID_DIFF/2)*EW_DIFF_1M
        ran += grid[i//interval_size]
        result[i:i+interval_size] += ran
    return result

In [15]:
'''
police_sampled = sampled_grids(police_grid)
police_sampled = pd.DataFrame(police_sampled, columns = ["위도", "경도"])
police_sampled.to_csv(data_path + "상계1위경도샘플.csv", index=False, encoding=encoding)
'''

In [12]:
police_sampled

,위도,경도
0,37.673399,127.054256
0,37.673447,127.054045
0,37.673578,127.054242
0,37.673156,127.054325
0,37.673209,127.054497
...,...,...
324,37.686257,127.055793
324,37.685699,127.055507
324,37.686189,127.055881
324,37.685953,127.055901


In [42]:
police_addr_data=pd.read_csv(data_path + "상계1샘플주소.csv", encoding=encoding)
police_addr_data.index=[i//(SAMPLING+1) for i in range(police_sampled.shape[0])]
police_addr_data["유형"] = None
police_addr_data["확률"] = None
police_addr_data

,위도,경도,지번주소,도로명주소,유형,확률
0,37.673399,127.054256,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None
0,37.673447,127.054045,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None
0,37.673578,127.054242,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None
0,37.673156,127.054325,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None
0,37.673209,127.054497,서울특별시 노원구 상계동 1044-2,서울특별시 노원구 동일로 1625,None,None
...,...,...,...,...,...,...
324,37.686257,127.055793,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None
324,37.685699,127.055507,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 누원로 18,None,None
324,37.686189,127.055881,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None
324,37.685953,127.055901,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None


In [38]:
'''
crime_loc = pd.read_csv(data_path+"범죄장소.csv", encoding='cp949')
crime_loc = crime_loc.groupby("장소")
crime_loc = crime_loc.sum()
crime_loc /= crime_loc.sum()
crime_loc.to_csv(data_path + "범죄장소확률.csv", encoding=encoding)
'''

In [43]:
crime_per = pd.read_csv(data_path+"범죄장소확률.csv", encoding=encoding)
crime_per

,장소,발생건수
0,고속도로,0.000454
1,공사장광산,0.003872
2,공장,0.000841
3,공지,0.000011
4,구금장소,0.000088
5,금융기관,0.007168
6,기타,0.180279
7,기타교통수단내,0.014701
8,노상,0.324340
9,단독주택,0.087457


In [44]:
DELAY_TIME = 1
map_url = "https://map.kakao.com/"

In [45]:
def openweb(mode="window"):
    if mode == "no":
        options = wb.ChromeOptions()
        options.add_argument('headless')
        driver = wb.Chrome(options=options)
    else:
        driver = wb.Chrome()
    driver.get(map_url)
    time.sleep(DELAY_TIME)
    return driver

In [46]:
def search(browser, address):
    search_box = browser.find_element(By.ID, "search.keyword.query")
    while search_box.get_attribute("value"):
        search_box.send_keys(Keys.CONTROL, 'a')
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)
    time.sleep(DELAY_TIME)

In [47]:
def find_postage(browser, num_addr):
    if "산" in num_addr:
        return "nozip"
    search(browser, num_addr)
    try:
        postage = browser.find_element(By.CLASS_NAME, "zip")
    except NoSuchElementException:
        return "nozip"
    else:
        return postage.text
    
def find_build_name(browser, road_addr, num_addr):
    search(browser, road_addr)
    try:
        buildname = browser.find_element(By.CLASS_NAME, "building")
    except NoSuchElementException:
        return find_postage(browser, num_addr)
    else:
        return buildname.text

In [50]:
browser = openweb("no")
for i in range(len(police_addr_data.index)):
    police_addr_data.iloc[i, 4] = find_build_name(browser, police_addr_data.iloc[i, 3], police_addr_data.iloc[i, 2])

In [51]:
police_addr_data.to_csv(data_path+"유형.csv",encoding=encoding)